In [19]:
import torch

import pytorch_lightning as pl

from src.config import get_parser
#from src.models.gan import GAN
from src.callback.points_image import PointsImage
from src.data.masked_datamodule import MaskedDataModule
from src.callback.export_mesh import ExportMesh
from pytorch3d.loss.chamfer import chamfer_distance

import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch3d.structures import Meshes
from pytorch3d.loss import ( 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from src.models.discriminator import Discriminator
from src.models.generator import Generator
from src.render.mesh_renderer import MeshPointsRenderer
from src.loss.edge_loss import EdgeLoss
from src.utilities.util import grid_to_list

import os
from random import randint

import numpy as np
import pytorch_lightning as pl
import torch
import trimesh
from src.utilities.util import make_faces, grid_to_list

In [38]:


w = h = config.data_blueprint_size
faces = make_faces(w, h)

config = get_parser().parse_args(args=[])
config.batch_size = 16
config.blueprint = 'blueprint_16_512.npz'
config.data_blueprint_size = 128
config.data_blueprint_coarse =  96
config.data_patch_size = 8
config.lr_g = 0.0003#0003
config.log_mesh_interval = 1

config.G_out_ch = 256 # 256 default

device =  torch.device("cuda")

dm = MaskedDataModule(config)
dm.setup()
dm

G = Generator(config).to(device) 
G

Generator(
  (head): ConvBlock(
    (conv): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Nonlinearity): Hardswish()
  )
  (body): Sequential(
    (b1): ConvBlock(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
    (b2): ConvBlock(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
    (b3): ConvBlock(
      (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
  )
  (tail): Sequential(

In [29]:
batch  = next(iter(dm.train_dataloader()))
points_coarse = batch['points_coarse']                
points_fine = batch['points']                
bs = points_fine.size(0)        
print(points_fine.shape)        

torch.Size([128, 3, 8, 8])


In [39]:
optimizer =  torch.optim.Adam(G.parameters())
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [ ]:
for epoch in range(10001): 
    for i, batch in enumerate(dm.train_dataloader()):
        points_coarse = batch['points_coarse'].to(device)
        points_fine = batch['points'].to(device)
        bs = points_fine.size(0)      

        optimizer.zero_grad()

        # train generator
        points_noise = points_coarse + torch.randn_like(points_coarse) * config.G_noise_amp            
        vertices = G(points_noise)
        #print(vertices.shape)
        #vrt_loss = F.mse_loss(vertices, points_noise.reshape_as(vertices))
        points_fine = points_fine.reshape_as(vertices)
        vrt_loss = F.l1_loss(vertices, points_fine)
        chm_loss, _ = chamfer_distance(vertices, points_fine)

        #vrt_loss = vrt_loss / bs
        #cos_sim = torch.cosine_similarity(normals, pt_normals, dim=-1)
        #normal_consistency_loss = -(cos_sim.sum() / cos_sim.numel() - 1.)

        #edge_loss = self.edge_loss(vertices)            
        g_loss = vrt_loss + chm_loss
        g_loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(epoch, i, g_loss.item())
    if epoch % 10 == 0:
        export_mesh(G, dm, faces, epoch, device)        

0 0 0.9061747193336487
0 100 0.19023287296295166
0 200 0.122894287109375
0 300 0.1398010402917862
0 400 0.09035624563694
1 0 0.06262992322444916
1 100 0.04245225340127945
1 200 0.05198286846280098
1 300 0.049091968685388565
1 400 0.029427379369735718
2 0 0.032173413783311844
2 100 0.02646406553685665
2 200 0.03190881013870239
2 300 0.0253318939357996
2 400 0.02191457524895668
3 0 0.051734767854213715
3 100 0.02709389477968216
3 200 0.024277955293655396
3 300 0.020067913457751274
3 400 0.021794991567730904
4 0 0.03830165043473244
4 100 0.021429529413580894
4 200 0.02527991682291031
4 300 0.02459024079144001
4 400 0.018067650496959686
5 0 0.029912292957305908
5 100 0.02123410254716873
5 200 0.021337691694498062
5 300 0.02312047779560089
5 400 0.021267011761665344
6 0 0.03493920713663101
6 100 0.021361013874411583
6 200 0.02443673461675644
6 300 0.01762765645980835
6 400 0.025585006922483444
7 0 0.019431427121162415
7 100 0.02418920211493969
7 200 0.018162937834858894
7 300 0.015471369959

60 300 0.008733467198908329
60 400 0.00951359048485756
61 0 0.011625397019088268
61 100 0.011804983019828796
61 200 0.007856874726712704
61 300 0.0093332938849926
61 400 0.008460871875286102
62 0 0.011005595326423645
62 100 0.00807764008641243
62 200 0.00737245287746191
62 300 0.008445490151643753
62 400 0.00964758638292551
63 0 0.007676273118704557
63 100 0.00759672187268734
63 200 0.008426502346992493
63 300 0.009455276653170586
63 400 0.011141791939735413
64 0 0.009547529742121696
64 100 0.007976186461746693
64 200 0.00950791034847498
64 300 0.009982896037399769
64 400 0.00869982410222292
65 0 0.009174742735922337
65 100 0.008574954234063625
65 200 0.008518449030816555
65 300 0.008925843052566051
65 400 0.008542370051145554
66 0 0.008523095399141312
66 100 0.009590843692421913
66 200 0.010863346047699451
66 300 0.009509594179689884
66 400 0.011392920278012753
67 0 0.008950311690568924
67 100 0.008350370451807976
67 200 0.008067803457379341
67 300 0.008625665679574013
67 400 0.007570

119 300 0.006385460961610079
119 400 0.007357697002589703
120 0 0.009199460037052631
120 100 0.008851072750985622
120 200 0.00704693328589201
120 300 0.008644821122288704
120 400 0.006787004414945841
121 0 0.006955644115805626
121 100 0.006355267018079758
121 200 0.008087009191513062
121 300 0.00947587937116623
121 400 0.007924114353954792
122 0 0.007120164576917887
122 100 0.006969484966248274
122 200 0.008060537278652191
122 300 0.007873998023569584
122 400 0.006558931898325682
123 0 0.008988060057163239
123 100 0.009500348940491676
123 200 0.00784982554614544
123 300 0.007071787025779486
123 400 0.006560247391462326
124 0 0.008783756755292416
124 100 0.00684834411367774
124 200 0.006784273777157068
124 300 0.00782998651266098
124 400 0.008402824401855469
125 0 0.007855519652366638
125 100 0.006792411208152771
125 200 0.006798818241804838
125 300 0.006438422482460737
125 400 0.008972421288490295
126 0 0.007044570054858923
126 100 0.007499538827687502
126 200 0.009775000624358654
126 

177 100 0.0066691236570477486
177 200 0.007504032459110022
177 300 0.007139559369534254
177 400 0.007834210060536861
178 0 0.0068258014507591724
178 100 0.0065017398446798325
178 200 0.006863536778837442
178 300 0.00626014731824398
178 400 0.006346888840198517
179 0 0.007829828187823296
179 100 0.006926217582076788
179 200 0.009052534587681293
179 300 0.006388337351381779
179 400 0.007213701494038105
180 0 0.007947629317641258
180 100 0.006166353821754456
180 200 0.008341959677636623
180 300 0.007240612059831619
180 400 0.006329333409667015
181 0 0.006903847213834524
181 100 0.00673915958032012
181 200 0.007194149773567915
181 300 0.006622819695621729
181 400 0.007906709797680378
182 0 0.0062384954653680325
182 100 0.00661122752353549
182 200 0.006420440971851349
182 300 0.006479848176240921
182 400 0.007362754084169865
183 0 0.011162404902279377
183 100 0.009233491495251656
183 200 0.007530128583312035
183 300 0.007872826419770718
183 400 0.0074576158076524734
184 0 0.0065550981089472

234 200 0.006934869568794966
234 300 0.005921804346144199
234 400 0.006244266871362925
235 0 0.006596492603421211
235 100 0.006113692652434111
235 200 0.0068534743040800095
235 300 0.0064674546010792255
235 400 0.006230994593352079
236 0 0.007112816907465458
236 100 0.0063018822111189365
236 200 0.0073867542669177055
236 300 0.00602663354948163
236 400 0.00696795666590333
237 0 0.005996112246066332
237 100 0.006048021838068962
237 200 0.00608807010576129
237 300 0.007073132786899805
237 400 0.006088182330131531
238 0 0.005796945188194513
238 100 0.006321797613054514
238 200 0.005962499417364597
238 300 0.0075781759805977345
238 400 0.007107360754162073
239 0 0.008587658405303955
239 100 0.008655448444187641
239 200 0.006163054145872593
239 300 0.005866898689419031
239 400 0.006117699667811394
240 0 0.00619534170255065
240 100 0.006393066141754389
240 200 0.006290061865001917
240 300 0.007182080298662186
240 400 0.006223272066563368
241 0 0.006406655069440603
241 100 0.00683333398774266

291 400 0.006685537751764059
292 0 0.007369468454271555
292 100 0.006741570308804512
292 200 0.00610047671943903
292 300 0.006644955836236477
292 400 0.007714932784438133
293 0 0.006698894314467907
293 100 0.0073247263208031654
293 200 0.005400680471211672
293 300 0.005770260933786631
293 400 0.007247488480061293
294 0 0.007423522882163525
294 100 0.008531906642019749
294 200 0.005648103542625904
294 300 0.0062161823734641075
294 400 0.007227602414786816
295 0 0.007197474595159292
295 100 0.006212295964360237
295 200 0.006716964300721884
295 300 0.006095859222114086
295 400 0.00592032540589571
296 0 0.005765191745012999
296 100 0.006653535179793835
296 200 0.007340619340538979
296 300 0.007054542191326618
296 400 0.0070607480593025684
297 0 0.0070471856743097305
297 100 0.005566906183958054
297 200 0.005558079108595848
297 300 0.005918754730373621
297 400 0.005911973770707846
298 0 0.00746867386624217
298 100 0.006038952153176069
298 200 0.0054636369459331036
298 300 0.0057417019270360

349 100 0.005987871903926134
349 200 0.0053395177237689495
349 300 0.005434646736830473
349 400 0.00609950115904212
350 0 0.006016682833433151
350 100 0.005944419652223587
350 200 0.005819073412567377
350 300 0.0061196074821054935
350 400 0.006804068572819233
351 0 0.006633829791098833
351 100 0.006420094054192305
351 200 0.007211317308247089
351 300 0.006543520838022232
351 400 0.005507690366357565
352 0 0.006142436061054468
352 100 0.006683533079922199
352 200 0.00653567211702466
352 300 0.007389031816273928
352 400 0.006201449315994978
353 0 0.0054907058365643024
353 100 0.006079100538045168
353 200 0.005286062601953745
353 300 0.007379789836704731
353 400 0.005090571939945221
354 0 0.0076227812096476555
354 100 0.006514673586934805
354 200 0.0067178113386034966
354 300 0.006020884495228529
354 400 0.005492481403052807
355 0 0.006760452874004841
355 100 0.005985892377793789
355 200 0.006560115143656731
355 300 0.006660946644842625
355 400 0.006197031587362289
356 0 0.005825038067996

In [17]:
vertices.shape

torch.Size([12, 4096, 3])

In [27]:
import os
from random import randint

import numpy as np
import pytorch_lightning as pl
import torch
import trimesh
from src.utilities.util import make_faces, grid_to_list

w = h = config.data_blueprint_size
faces = make_faces(w, h)
    
current_epoch = 0


def export_mesh(G, dm, faces, current_epoch, device):
    points = dm.train_dataloader().dataset.points_coarse
    
    i = randint(0, points.size(0) -1)
    pts = points[i][None].to(device)
    with torch.no_grad():
        G.eval()
        vertices = G(pts)[0].cpu().numpy()
        mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
        mesh_dir = os.path.join('./data', 'mesh')
        if not os.path.exists(mesh_dir):
            os.makedirs(mesh_dir)
        file_path = os.path.join(mesh_dir, f'mesh_{current_epoch}_{i}.stl')
        mesh.export(file_path)
        G.train()

In [ ]:
# points = trainer.datamodule.train_dataloader().dataset.points.to(pl_module.device)
        batch  = next(iter(trainer.datamodule.train_dataloader()))
        points = batch['points_coarse'].to(pl_module.device)
        # points = trainer.datamodule.train_dataloader().dataset.points_coarse#
        if self.faces is None:
            print('points.size(-2), points.size(-1)', points.size(-2), points.size(-1))
            self.faces = make_faces(points.size(-2), points.size(-1))
        # generate images
        with torch.no_grad():
            pl_module.eval()
            i = randint(0, points.size(0) -1)
            pts = points[i][None].to(pl_module.device)          
            vertices = pl_module.G(pts)[0].cpu().numpy()
            print('on_epoch_end vertices.shape:', vertices.shape)
            mesh = trimesh.Trimesh(vertices=vertices, faces=self.faces)
            mesh_dir = os.path.join(trainer.log_dir, 'mesh')
            if not os.path.exists(mesh_dir):
                os.makedirs(mesh_dir)
            file_path = os.path.join(mesh_dir, f'mesh_{trainer.current_epoch}_{i}.stl')
            mesh.export(file_path)            
            if trainer.current_epoch == 0: 
                vertices = grid_to_list(pts)[0].cpu().numpy()
                mesh = trimesh.Trimesh(vertices=vertices, faces=self.faces)
                mesh_dir = os.path.join(trainer.log_dir, 'mesh')
                if not os.path.exists(mesh_dir):
                    os.makedirs(mesh_dir)
                file_path = os.path.join(mesh_dir, f'mesh_{trainer.current_epoch}_{i}_orig.stl')
                mesh.export(file_path)

            pl_module.train()

In [23]:
G.

Generator(
  (head): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Nonlinearity): Hardswish()
  )
  (body): Sequential(
    (b1): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
    (b2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
    (b3): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Nonlinearity): Hardswish()
    )
  )
  (tail): Sequential(
    (0): C